In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import gc

In [3]:
data_path = "../../allData/validationData/out_7day_test/test_sessions.jsonl"

In [6]:
%%time
test_df = pd.DataFrame()
chunks = pd.read_json(data_path, lines=True, chunksize=100_000)

for chunk in chunks:
    event_dict = {'session': [],'aid': [],'ts': [],'type': []}
    for session, events in zip(chunk['session'].tolist(), chunk['events'].tolist()):
        for event in events:
            event_dict['session'].append(session)
            event_dict['aid'].append(event['aid'])
            event_dict['ts'].append(event['ts'])
            event_dict['type'].append(event['type'])
    chunk_session = pd.DataFrame(event_dict)
    test_df = pd.concat([test_df, chunk_session])
            
test_df = test_df.reset_index(drop=True)
test_df

CPU times: user 34.7 s, sys: 1.13 s, total: 35.9 s
Wall time: 36 s


,session,aid,ts,type
0,11098528,11830,1661119200060,clicks
1,11098529,1105029,1661119200259,clicks
2,11098530,264500,1661119200974,clicks
3,11098530,264500,1661119288407,clicks
4,11098530,409236,1661119369986,clicks
...,...,...,...,...
7580963,12899774,33035,1661723968869,clicks
7580964,12899775,1743151,1661723970935,clicks
7580965,12899776,548599,1661723972537,clicks
7580966,12899777,384045,1661723976974,clicks


In [7]:
%%time
test_df["minutes"] = test_df[["session", "ts"]].groupby("session").diff(-1)*(-1/1000/60)
test_df = test_df.sort_values(['minutes'],ascending=False)

test_action_df = test_df.copy()
test_action_df.aid = ' ' + test_df.aid.astype('str')
test_action_df = test_action_df.groupby(['session','type'])['aid'].sum().reset_index()
test_action_df

CPU times: user 3min 45s, sys: 1min 16s, total: 5min 2s
Wall time: 5min 33s


,session,type,aid
0,11098528,clicks,11830
1,11098529,clicks,1105029
2,11098530,carts,409236
3,11098530,clicks,409236 409236 264500 264500 409236
4,11098531,clicks,1365569 1553691 1271998 1309633 452188 624163...
...,...,...,...
2086659,12899774,clicks,33035
2086660,12899775,clicks,1743151
2086661,12899776,clicks,548599
2086662,12899777,clicks,384045


In [8]:
next_orders_df = pd.DataFrame(test_action_df.loc[(test_action_df["type"] == 'carts'), ])
next_orders_df['type'] = 'orders'
next_orders_df

,session,type,aid
2,11098530,orders,409236
7,11098533,orders,1074173 1309900
10,11098535,orders,745365
13,11098537,orders,1409748 1409748 358965
16,11098538,orders,1711586
...,...,...,...
2086483,12899601,orders,946970
2086495,12899613,orders,983198
2086514,12899631,orders,1055835
2086528,12899644,orders,436912 822934


In [9]:
next_carts_df = pd.DataFrame(test_action_df.loc[(test_action_df["type"] == 'clicks'), ])
next_carts_df['type'] = 'carts'
next_carts_df

,session,type,aid
0,11098528,carts,11830
1,11098529,carts,1105029
3,11098530,carts,409236 409236 264500 264500 409236
4,11098531,carts,1365569 1553691 1271998 1309633 452188 624163...
6,11098532,carts,7651 876469
...,...,...,...
2086659,12899774,carts,33035
2086660,12899775,carts,1743151
2086661,12899776,carts,548599
2086662,12899777,carts,384045


In [10]:
next_clicks_df = pd.DataFrame(test_action_df.loc[(test_action_df["type"] == 'clicks'), ]).copy()

In [11]:
recommend_df = pd.concat([next_orders_df, next_carts_df, next_clicks_df], axis =0)
recommend_df["session_type"] = recommend_df["session"].astype('str') + "_" + recommend_df["type"] 
recommend_df

,session,type,aid,session_type
2,11098530,orders,409236,11098530_orders
7,11098533,orders,1074173 1309900,11098533_orders
10,11098535,orders,745365,11098535_orders
13,11098537,orders,1409748 1409748 358965,11098537_orders
16,11098538,orders,1711586,11098538_orders
...,...,...,...,...
2086659,12899774,clicks,33035,12899774_clicks
2086660,12899775,clicks,1743151,12899775_clicks
2086661,12899776,clicks,548599,12899776_clicks
2086662,12899777,clicks,384045,12899777_clicks
